## Bibliotecas

In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

import os
import csv

## Configurações

In [ ]:
dataset_path = "/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/dataset"  # pasta principal com subpastas das classes
model_path = "/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/notebooks/trained/v1_10ep.pth"  # caminho do modelo treinado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
output_csv = "/home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/features_dataset.csv"
num_classes = 30

## Carregar a ResNet50 pré-treinada

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

model.fc = torch.nn.Linear(model.fc.in_features, num_classes) # seta as classes de peixes

model.load_state_dict(torch.load(model_path, map_location=device))

model.eval()
model.to(device)


feature_extractor = nn.Sequential(*list(model.children())[:-1]) # seta para geração de embeddings na última camada fully-connected (2048D)
feature_extractor.eval()

## Transform

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## Geração da base de dados de imagens em um CSV

In [12]:
with open(output_csv, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # Cabeçalho
    writer.writerow(["id_classe", "caminho_imagem", "nome_classe", "vetor_caracteristicas"])

    for idx, class_name in enumerate(sorted(os.listdir(dataset_path))):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue  # ignora arquivos não-pasta

        for img_name in os.listdir(class_path):
            if not img_name.lower().endswith((".png", ".jpg", ".jpeg")):
                continue  # ignora arquivos não-imagem

            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path).convert("RGB")
            img_tensor = transform(img).unsqueeze(0).to(device)

            with torch.no_grad():
                feats = feature_extractor(img_tensor)
                feats = feats.view(feats.size(0), -1)  # achatar
                feats_np = feats.cpu().numpy().flatten()  # vetor 1D

            # Salvar no CSV
            writer.writerow([idx, img_path, class_name, " ".join(map(str, feats_np))])

print(f"Extração concluída! CSV salvo em: {output_csv}")

Extração concluída! CSV salvo em: features_dataset.csv
